# eBay Items Scraping and Databasing - Stella (Sute) Li

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
import re
import time
import mysql.connector
import pandas as pd

## Store the link to items in two separate text files for sponsored and non-sponsored items

In [211]:
# specifying search key word = playstation 4 slim, buy-it-now items only, items per page = 100
url = 'https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&LH_BIN=1&_ipg=100'

In [212]:
# create two files to store all the links
file_sponsored = open('sponsored.txt', 'w')
file_nonsponsored = open('non-sponsored.txt', 'w')

for i in range(1,11):
    # parse the webpages and find sponsored items
    url2 = url + '&_pgn=' + str(i)
    html = requests.get(url2)
    soup = BeautifulSoup(html.text, "html.parser")

    pattern = re.compile(r'.*[S].*[P].*[O].*[N].*[S].*[O].*[R].*[E].*[D].*')
    
    # find urls of sponsored and not sponsored items and write into the files
    for elem in soup.find_all('div', attrs = {'style':'display:flex;'}):
        text = elem.getText()
        if pattern.search(text):
            sponsored_url = elem.parent.get('href')
            file_sponsored.write(sponsored_url + '\n')
            
        if pattern.search(text) == None:
            unsponsored_url = elem.parent.get('href')
            file_nonsponsored.write(unsponsored_url + '\n')
        
            
file_sponsored.close()
file_nonsponsored.close()

## Save the webpages into two separate folders

In [213]:
# create the two folders 'sponsored' and 'non-sponsored'
cwd = os.getcwd()
path_sponsored = os.path.join(cwd, r'sponsored')
path_nonsponsored = os.path.join(cwd, r'non-sponsored')
try:
    os.mkdir(path_sponsored)
    os.mkdir(path_nonsponsored)
except:
    print('directory already exists')

In [214]:
# store html code of webpages
def webpage_to_file(url):
    request = requests.get(url)
    html = request.text
    return html

In [215]:
# how to find the id of items from the url
id_pattern = re.compile('/\d{12}\?')

# save all the webpages from the links
def save_ebay_pages(sponsored = False):
    counter = 0
    #  opens the two files and downloads each of the pages (URLs) into the folders "sponsored" and "non-sponsored"
    if sponsored == True:
        file_name = 'sponsored.txt'

    else:
        file_name = 'non-sponsored.txt'        
        
    with open(file_name, 'r') as file:
        if sponsored == True:
            os.chdir(path_sponsored)
        else:
            os.chdir(path_nonsponsored)
            
        for line in file.readlines():
            counter +=1
            url = line
            item_id = re.search(id_pattern, line).group()[1:-1]
            
            try:
                html = webpage_to_file(url)
            # error handling    
            except:
                print('error at link no.', counter, ': \n', url)
                time.sleep(2)
                html = webpage_to_file(url)
                continue
            # name the files with item ID    
            ebay_html = open(item_id +'.htm', 'w', encoding='utf8')
            ebay_html.write(html)
            ebay_html.close()
                
            time.sleep(2) # 2-second pause between requests       

In [216]:
save_ebay_pages(sponsored=True)

In [217]:
os.chdir("..")

In [218]:
save_ebay_pages(sponsored=False)

## Scrape a few pieces of information
seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).

In [237]:
# navigate to the path of non-sponsored items
def navigate_to_path_nonsponsored():
    if os.getcwd() != os.chdir(path_nonsponsored):
        os.chdir(path_nonsponsored)

In [238]:
# navigate to the path of sponsored items
def navigate_to_path_sponsored():
    if os.getcwd() != os.chdir(path_sponsored):
        os.chdir(path_sponsored)

In [671]:
navigate_to_path_sponsored()

In [673]:
navigate_to_path_nonsponsored()

In [684]:
# store all items in a list of dictionaries
# initiate the list
ebay_items = []
index = 0

In [491]:
# to find the price of items
price_pattern = re.compile('\$[\d\.,]+')

In [685]:
def store_ebay_items(sponsored=False):
    global index
    
    if sponsored == True:
        navigate_to_path_sponsored()
    else:
        navigate_to_path_nonsponsored()
        
    for filename in os.listdir():            
        f = open(filename, 'r', encoding = 'ISO-8859-1')
        html = f.read()
        soup = BeautifulSoup(html, "html.parser")
        
        # the id of the item read from the filename
        item_id = filename[:-4]
        # append a dictionary for each item to the list and update the id for the item
        ebay_items.append({'item_id': item_id})
        # update whether the item is sponsored
        if sponsored == True:
            ebay_items[index].update({'sponsored': True})
        else:
            ebay_items[index].update({'sponsored': False})

        try:
            #### seller name ####
            seller_name = soup.find('span', class_='mbg-nw').text
            ebay_items[index].update({'seller_name': seller_name})

            #### seller score ####
            seller_score = soup.find('span', class_ = 'mbg-l').find('a').text
            ebay_items[index].update({'seller_score': seller_score})

            #### number of items sold ####
            num_items_sold_object = soup.find('a', class_ = 'vi-txt-underline')
            if num_items_sold_object == None:
                num_items_sold = 0
            else:
                num_items_sold = num_items_sold_object.text[:num_items_sold_object.text.index(' ')].replace(',', '')
            ebay_items[index].update({'num_items_sold': num_items_sold})

            #### whether best offer option is available ####
            best_offer_object = soup.find('div', class_ = 'vi-bbox-dspn u-flL lable boLable')
            if best_offer_object == None:
                best_offer_available = False
            else:
                best_offer_available = True
            ebay_items[index].update({'best_offer_available': best_offer_available})

            #### title ####
            title = soup.find('h1', class_ = 'it-ttl').text[len('Details about  \xa0'):]
            ebay_items[index].update({'title': title})

            #### whether returns are allowed ####
            returns_text = soup.find('span', id = 'vi-ret-accrd-txt').text
            if 'Seller does not accept returns' in returns_text:
                returns_allowed = False
            else:
                returns_allowed = True
            ebay_items[index].update({'returns_allowed': returns_allowed})

            #### condition ####
            condition = soup.find('div', id = 'vi-itm-cond').text
            ebay_items[index].update({'condition': condition})

            #### item price ####
            item_price = soup.find('span', id = 'prcIsum').text
            if item_price.startswith('US') == False:
                item_price = soup.find('span', id = 'convbidPrice').text
            if item_price.startswith('US') == False:
                item_price = soup.find('span', id = 'convbinPrice').text

            #### shipping cost ####
            shipping_cost = soup.find('span', id = 'fshippingCost').contents[1].text
            if shipping_cost.startswith('$') == False and shipping_cost != 'FREE':
                shipping_cost = soup.find('span', id = 'convetedPriceId').text

        # item price and shipping cost on different pages are in different formats
        except:
            try:
                item_price = soup.find('span', class_ = 'cc-text-spans--BOLD').text
                if item_price.startswith('US') == False:
                    item_price = soup.find('span', id = 'convbidPrice').text
                if item_price.startswith('US') == False:
                    item_price = soup.find('span', id = 'convbinPrice').text
                shipping_cost = soup.find('a', id = 'e3').text

            except:
                try:
                    item_price = soup.find('span', {'itemprop': 'price'}).text
                    if item_price.startswith('US') == False:
                        item_price = soup.find('span', id = 'convbidPrice').text
                    if item_price.startswith('US') == False:
                        item_price = soup.find('span', id = 'convbinPrice').text
                    shipping_cost = soup.find('strong', class_ = 'sh_gr_bld').text
                except:
                    try:
                        item_price = soup.find('span', id = 'convbinPrice').text
                        if soup.find('span', id = 'fShippingSvc').contents[1].text != None:
                            shipping_cost = soup.find('span', id = 'fShippingSvc').contents[1].text
                        else:
                            shipping_cost = soup.find('span', id = 'fShippingSvc').text
                    except:
                        shipping_cost = None    
        # after locating the item price, convert it into a clean format
        price = re.search(price_pattern, item_price).group()[1:].replace(',','')

        # clean the shipping costs
        if shipping_cost != None:
            if shipping_cost.startswith('$'):
                shipping = shipping_cost.replace('$','')
                shipping_computed = True
            elif 'free' in shipping_cost.lower():
                shipping = 0
                shipping_computed = True
            else:
                shipping = None
                shipping_computed = False
        else:
            shipping = None
            shipping_computed = False

        ebay_items[index].update({'item_price': price})
        ebay_items[index].update({'shipping_cost': shipping})
        ebay_items[index].update({'shipping_computed': shipping_computed})

        f.close()
        index +=1

In [687]:
store_ebay_items(sponsored = True)

In [688]:
store_ebay_items(sponsored = False)

## Connect to and store all information above into a SQL database

If an item misses ANY of the information in (d), you should insert that missing value as NULL into the table. Convert any price (item price and shipping price) into a "dollar-cent" format (e.g., convert 12.34 into 1234 and 12 into 1200. If an item does not include cents in the price, insert zeros.) and insert the price as INT into the table.

In [631]:
# connect to my mysql database
mydb = mysql.connector.connect(host='localhost', user='root', connect_timeout=5)

In [632]:
cursor = mydb.cursor()

In [633]:
# create the database and table
cursor.execute('CREATE DATABASE eBay')

In [692]:
cursor.execute('USE eBay')
cursor.execute('DROP TABLE IF EXISTS eBay_items')

In [693]:
create_table_script = ('CREATE TABLE eBay_items ( '
'item_id BIGINT, '
'sponsored BIT, '
'seller_name VARCHAR(255), '
'seller_score INTEGER, '
'num_items_sold INTEGER, '
'best_offer_available BIT, '
'title VARCHAR(255), '
'returns_allowed BIT, '
'item_condition VARCHAR(255), '
'item_price INTEGER, '
'shipping_cost INTEGER, '
'shipping_computed BIT);')

In [694]:
cursor.execute(create_table_script)

In [659]:
insert_script = ('INSERT INTO eBay_items (item_id, sponsored, seller_name, seller_score,'
                                            'num_items_sold, best_offer_available, title,' 
                                            'returns_allowed, item_condition, item_price, shipping_cost,'
                                            'shipping_computed) '
            'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)')

In [698]:
# insert each item into the table
for elem in ebay_items:
    # find and create all the corresponding variables
    item_id = elem['item_id']
    sponsored = 1 if elem['sponsored']==True else 0
    seller_name = elem['seller_name']
    seller_score = elem['seller_score']
    num_items_sold = elem['num_items_sold']
    best_offer_available = 1 if elem['best_offer_available']==True else 0
    title = elem['title']
    returns_allowed = 1 if elem['returns_allowed']==True else 0
    item_condition = elem['condition']
    item_price = int(float(elem['item_price'])*100)
    shipping_cost = None if (elem['shipping_cost']==None) else int(float(elem['shipping_cost'])*100)
    shipping_computed = 1 if elem['shipping_computed']==True else 0
    
    values = (item_id, sponsored, seller_name, seller_score, num_items_sold, best_offer_available, title,
          returns_allowed, item_condition, item_price, shipping_cost, shipping_computed)
    # insert each item into the table
    cursor.execute(insert_script, values)
    mydb.commit()

## Some summay statistics of the items
The mean, min, max, and sd for each column, grouped by "sponsor/non-sponsor" and "condition" (group by at the same time, not separately). For binary categorical columns, use 0-1 conversion. For e.g., for the "returns allowed" convert YES to 1 and NO to 0 and then calculate the stats. If it is NOT a numerical/binary categorical column, print to the screen the count of each category level.

In [716]:
# compile the query to find summary statistics
summary_query = ('SELECT sponsored, item_condition, '
'avg(seller_score), max(seller_score), min(seller_score), stddev(seller_score),'
'avg(num_items_sold), max(num_items_sold), min(num_items_sold), stddev(num_items_sold),'
'avg(item_price), max(item_price), min(item_price), stddev(item_price), '
'avg(shipping_cost), max(shipping_cost), min(shipping_cost), stddev(shipping_cost),'
'avg(best_offer_available), max(best_offer_available), min(best_offer_available), stddev(best_offer_available),'
'avg(returns_allowed), max(returns_allowed), min(returns_allowed), stddev(returns_allowed),'
'count(item_condition) '
'FROM ebay_items '
'GROUP BY sponsored, item_condition;')

In [718]:
cursor.execute(summary_query)

In [719]:
result = cursor.fetchall()

In [726]:
# store the summary statistics into a table and convert into a pandas dataframe
summary_table = []

In [727]:
for row in result:
    summary_table.append(row)

In [730]:
# update the column names
summary_df = pd.DataFrame(summary_table)
summary_df.columns = ['sponsored', 'item_condition', 'avg(seller_score)', 'max(seller_score)', 'min(seller_score)', 'stddev(seller_score)',
'avg(num_items_sold)', 'max(num_items_sold)', 'min(num_items_sold)', 'stddev(num_items_sold)',
'avg(item_price)', 'max(item_price)', 'min(item_price)', 'stddev(item_price)',
'avg(shipping_cost)', 'max(shipping_cost)', 'min(shipping_cost)', 'stddev(shipping_cost)',
'avg(best_offer_available)', 'max(best_offer_available)', 'min(best_offer_available)', 'stddev(best_offer_available)',
'avg(returns_allowed)', 'max(returns_allowed)', 'min(returns_allowed)', 'stddev(returns_allowed)',
'count(item_condition)']

In [736]:
# to be able to view all the columns
pd.set_option('display.max_columns', 100)

In [741]:
round(summary_df.sort_values(by = ['sponsored', 'item_condition'], ascending = False),2)

,sponsored,item_condition,avg(seller_score),max(seller_score),min(seller_score),stddev(seller_score),avg(num_items_sold),max(num_items_sold),min(num_items_sold),stddev(num_items_sold),avg(item_price),max(item_price),min(item_price),stddev(item_price),avg(shipping_cost),max(shipping_cost),min(shipping_cost),stddev(shipping_cost),avg(best_offer_available),max(best_offer_available),min(best_offer_available),stddev(best_offer_available),avg(returns_allowed),max(returns_allowed),min(returns_allowed),stddev(returns_allowed),count(item_condition)
0,1,Used,17273.4242,258795,0,44379.23,26.8788,1690,0,206.42,26227.2121,55351,15000,7805.77,408.3714,4090,0,913.99,0.5303,1,0,0.50,0.5303,1,0,0.50,132
3,1,Seller refurbished,15842.0000,41451,0,15258.84,348.2000,1737,0,694.40,22813.6000,25300,19999,1943.15,0.0000,0,0,0.00,0.4000,1,0,0.49,0.8000,1,0,0.40,10
4,1,Open box,15819.6000,64480,5,24974.39,1.0000,5,0,2.00,26060.0000,33500,21000,4337.56,750.0000,1500,0,750.00,0.6000,1,0,0.49,0.4000,1,0,0.49,10
2,1,New,34961.4130,916791,0,155293.22,13.8478,313,0,48.61,32298.6087,57999,22100,8818.45,402.8824,4999,0,1181.47,0.2609,1,0,0.44,0.7174,1,0,0.45,92
1,1,Manufacturer refurbished,294.5000,533,56,238.50,1.5000,3,0,1.50,26949.5000,33900,19999,6950.50,0.0000,0,0,0.00,1.0000,1,1,0.00,0.5000,1,0,0.50,4
5,0,Used,51318.7711,224421,0,74776.60,0.1295,17,0,1.23,22654.9437,50000,8500,5084.40,407.4164,4660,0,840.60,0.4128,1,0,0.49,0.5009,1,0,0.50,1066
9,0,Seller refurbished,312811.7500,2265043,0,742315.99,48.6563,1316,0,228.47,23092.0000,36995,11200,5174.40,278.2083,2999,0,762.45,0.3125,1,0,0.46,0.5938,1,0,0.49,64
8,0,Open box,15175.7759,568351,0,77237.71,0.1724,5,0,0.77,26350.1207,52995,12500,7123.89,758.5882,5700,0,1483.86,0.5517,1,0,0.50,0.2069,1,0,0.41,116
6,0,New,13088.1786,638464,-1,47683.85,17.3681,1896,0,140.30,31965.2665,126999,14300,11013.31,327.7580,4854,0,884.55,0.3077,1,0,0.46,0.5110,1,0,0.50,728
7,0,Manufacturer refurbished,453019.2000,2265041,0,906010.90,1.0000,5,0,2.00,21359.8000,25000,19000,2148.94,0.0000,0,0,0.00,0.8000,1,0,0.40,0.4000,1,0,0.49,10


## How sponsored and non-sponsored items appear to be different

- sponsored items tend to have a higher seller score for new items, and lower seller score for items in all other conditions.
- sponsored items also have more items sold on average, especially for seller refurbished items.
- on average, sponsored items have higher item prices for new, used, manufacturer refurbished items, and lower price for items in other conditions.
- on average, more sponsored items provide the option of best offers and allow returns.
- the best variable that could be predict if the item was sponsored or not could be best offer available and returns allowed. Also, seller score in conjunction with item condition would could be a predictor. High seller score for new items would indicate an item to be sponsored.
